In [1]:
import numpy as np
import gurobipy as gp

# Simple LP vs IP example

In [ ]:
obj = np.array([5,4])
A = np.zeros((2,2))
A[0,:] = [1,1]
A[1,:] = [10,6]
sense = ['<','<']
b = np.array([5,45])

In [ ]:
lpMod = gp.Model()
lpMod_x = lpMod.addMVar(2)
lpMod_con = lpMod.addMConstrs(A, lpMod_x, sense, b)
lpMod.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE)

lpMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
lpMod.optimize()

In [ ]:
lpMod_x.x

In [ ]:
ipMod = gp.Model()
ipMod_x = ipMod.addMVar(2,vtype=['I','I']) # 'C' or 'I' or 'B'
ipMod_con = ipMod.addMConstrs(A, ipMod_x, sense, b)
ipMod.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE)

ipMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
ipMod.optimize()

In [ ]:
ipMod_x.x

# NPV Example

In [ ]:
obj = np.array([16000,8000,10000,19500,22000,12000,7500])
sense = ['<']
A = np.zeros((1,7))
A[0,:] = [5000,2500,3500,6000,7000,4500,3000]
b = np.array([15000])


In [ ]:
npvMod = gp.Model()
npvMod_x = npvMod.addMVar(len(obj),vtype=['B']*len(obj)) # 'C' or 'I' or 'B'
npvMod_con = npvMod.addMConstrs(A, npvMod_x, sense, b)
npvMod.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE)

npvMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
npvMod.optimize()

In [ ]:
npvMod_x.x

In [ ]:
npvMod.objVal

In [ ]:
npvMod_con[0].Pi

# Buy in Problem

In [2]:
# buy-in problem
# variables are x1, x2, ..., x5, y1, y2, ..., y5
M = 10
obj = np.array([10,18,11,9,10,-25,-35,-28,-20,-40]) # profits per unit and buy in costs
ub = np.array([5,4,5,7,3,1,1,1,1,1]) # upper bound on each project
A = np.zeros((6,10))
b = np.zeros(6)
sense = ['<']*6
A[0,:] = [5,7,6,4,8,25,35,28,20,40] # budget constraint
b[0] = 125

A[1,[0,5]] = [1,-M] # big M constraints
A[2,[1,6]] = [1,-M]
A[3,[2,7]] = [1,-M]
A[4,[3,8]] = [1,-M]
A[5,[4,9]] = [1,-M]
vtype = ['C']*5 + ['B']*5


In [3]:
biMod = gp.Model()
biMod_x = biMod.addMVar(len(obj),vtype=vtype,ub=ub) # vtype can be: 'C' or 'I' or 'B'
biMod_con = biMod.addMConstrs(A, biMod_x, sense, b)
biMod.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE)

biMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
biMod.optimize()

Academic license - for non-commercial use only - expires 2022-08-22
Using license file /Users/dan/gurobi.lic


In [4]:
biMod_x.x

array([ 0.,  4.,  0.,  7.,  0.,  0.,  1., -0.,  1., -0.])

In [ ]:
# y2 and y4 are 1
# y1, y3 and y5 are 0
# so only x2 and x4 are allowed to be positive
# x1, x3, and x5 must be 0, which they are!
# buy 4 units of investment 2 and 7 units of investment 4
biMod.objVal # this is the PROFIT, not the payoff
# we don't need to include money not invested because you can't get a PROFIT on that money
# you actually only invest $101, not your whole $125 budget...
# what if you could take all money not invested in a project and put it in a bank
# account that pays 1% interest?
# then you could also include a slack variable for  all money not spent 
# in the budget constraint.  then that could be added to the objective
# (just its interest) so interest on money not invested gets included in profits
# can you set up the model to include that?
